<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages-Import" data-toc-modified-id="Packages-Import-1">Packages Import</a></span></li><li><span><a href="#plot-ciCOH-distributions" data-toc-modified-id="plot-ciCOH-distributions-2">plot ciCOH distributions</a></span></li><li><span><a href="#identify-threshold" data-toc-modified-id="identify-threshold-3">identify threshold</a></span><ul class="toc-item"><li><span><a href="#sinu-func" data-toc-modified-id="sinu-func-3.1">sinu func</a></span></li><li><span><a href="#Normal-Distribution" data-toc-modified-id="Normal-Distribution-3.2">Normal Distribution</a></span></li></ul></li><li><span><a href="#Test-Part" data-toc-modified-id="Test-Part-4">Test Part</a></span></li></ul></div>

## Packages Import

In [ ]:
import os, sys
import numpy as np
from numpy.random import normal
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt
plt.rc('text', usetex = True)

In [ ]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule


from simulated.simulated_timeseries import gen_ciCOH_population_sin

## plot ciCOH distributions

In [ ]:
def _plot_ciCOH_distributions(ciCOHs, text_title = "ciCOH distribution"):
    """
    
        inputs:
            ciCOHs: a vector of ciCOH values (ntimes, )
    """
    
    

    # Plot the histogram.
    plt.hist(ciCOHs, bins=10, density=True, alpha=0.6, color='g')

    # Fit a normal distribution to the data:
    mu, std = norm.fit(ciCOHs)

    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)


    # lot the 2*std lines
    plt.plot([mu + 2 * std, mu + 2 * std], np.array(plt.ylim())/2, 'b--', 
             [mu - 2 * std, mu - 2 * std], np.array(plt.ylim())/2, 'b--')

    # plot the 3*std lines
    plt.plot([mu + 3 * std, mu + 3 * std], np.array(plt.ylim())/2, 'k--', 
             [mu - 3 * std, mu - 3 * std], np.array(plt.ylim())/2, 'k--')



    
    # Plot statistical analysis:  mu, std and 95% 
    
    # the content of statistical analysis text
    eq1 = r"\begin{eqnarray*}" + \
      r"\mu = " + str(np.around(mu, decimals=2)) + "\\"\
      r"\std = " + str(np.around(std, decimals=4)) + "\\"\
      r"\95 \%= " + str(np.around(mu + 2*std, decimals=4)) + "\\"\
      r"\99 \%= " + str(np.around(mu + 3*std, decimals=4)) +\
      r"\end{eqnarray*}"
    
    # the plot position of mu, std, and 95% text
    ymin, ymax = plt.ylim()
    xmin, xmax = plt.xlim()
    pos_x = xmin + (xmax - xmin)* 4/5
    pos_y = ymin + (ymax -ymin) * 4/5
    plt.text(pos_x, pos_y, eq1, {'fontsize': 12}, va="top", ha="left")
    
    
    # Plot title
    plt.title(text_title)

    
    plt.show()

## identify threshold

###  sinu func

In [ ]:
def threshold_ciCOH_sin(ntimes, ntrials, ntemp, f, t = 1, alpha = 0.05, ploton = True):
    
        """
        using sinc function to simulated the no connections and identify the threshold for has connection
        
        @paras:
            ntimes: the repeated time (can be set nchns * nchns)
            
            ntrials: the number of trials 
            
            ntemp: the length of the temporal data
            
            f: the frequency of the two time series (Hz)
            
            t: the total time duration for the time series (default 1s)
            
            alpha: the critirial (5%, or 1%)
            
            ploton: show a figure if True
        
        @return:
            threshold: the ciCOH threshold value (a scalar)
            
            
        Example Usage:
        
            thres = threshold_ciCOH_sin(ntimes = 1000, ntrials = 100, ntemp = 500, f = 30, t = 1, alpha = 0.01, ploton = True)
            
        """
        
        print("identifying the ciCOH threshold using sinc.....")
        
        # generate the ciCOH population 
        ciCOHs = gen_ciCOH_population_sin(ntimes = ntimes, ntrials = ntrials, ntemp = ntemp, f = f, t = t, Desired_SNR_dB = 20)
        
        
        
        ### Identify the threshold
        
        mu, std = norm.fit(ciCOHs) # Fit a normal distribution to the data
        if alpha == 0.05:
            threshold = np.around(mu + 2*std, decimals=2)
        
        elif alpha == 0.01:
            threshold = np.around(mu + 3*std, decimals=2)

        
        
        
        ### plot the ciCOH distribution of ciCOHs of the simulated data 
        if ploton:
            text_title = "ntrials = " +  str(ntrials) + ", ntemp = " + str(ntemp) +\
                        ",f = "  + str(f) + "Hz, ntimes = " + str(ntimes)
            
            _plot_ciCOH_distributions(ciCOHs, text_title = text_title)
            
            
    
        print("threshold = " + str(threshold) + ", mu = " + str(mu) + ", std = " + str(std))
            
            
        return threshold, mu, std

In [ ]:
## Multiple comparison correction
def corr_threshold_ciCOH_sin_BH(ciCOHs_actual, ntimes, ntrials, ntemp, false_rate = 0.25, mu = None, std = None, f = 10, t = 1):
    """
    
        control the false discovery reate using Benjamini-Hochberg procedure
        
        
        @paras:
            
            ciCOHs_actual:the ciCOH values (nvalues, )
            
            ntimes: the repeated time (can be set nchns * nchns)
            
            ntrials: the number of trials 
            
            ntemp: the length of the temporal data
            
            mu, std: the mean and std values of the null hypothesis no connection probability distribution
            
            f: the frequency of the two time series (Hz)
            
            t: the total time duration for the time series (default 1s)
            
            
            
        ref:
            http://www.biostathandbook.com/multiplecomparisons.html
        
    """
    
            
    print("identifying the ciCOH corrected threshold using sinc and Benjamini-Hochberg procedure....")

    if mu is None or std is None:
    
        # generate the ciCOH population 
        ciCOHs_simulated = gen_ciCOH_population_sin(ntimes = ntimes, ntrials = ntrials, ntemp= ntemp, f = f, t = 1, Desired_SNR_dB = 20)

        # evaluate the null hypothesis distribution mean and std using Normal distribution
        mu, std = norm.fit(ciCOHs_simulated)

    
    
    ### BH corrected
    
    ciCOHs_actual = abs(ciCOHs_actual)
    
    # calculate the pvalue for each abs(ciCOH) value
    pvalues = stats.norm.sf(ciCOHs_actual, loc = mu, scale = std) * 2
    
    # sort the pvalues
    ind = np.argsort(pvalues)
    pvalues_sorted = pvalues[ind]
    ciCOHs_sorted = ciCOHs_actual[ind];
    
    # generate the Benjamini-Hochberg critical values
    criticalvalue_bh = np.array([*range(1, len(pvalues) + 1, 1)])/len(pvalues) * false_rate

    # find the index which has the largest P value what pvalue < criticalvalue_bh
    for i in range(len(pvalues_sorted)):
        if(pvalues_sorted[i] >= criticalvalue_bh[i]):
            break
    
    
    
    corrected_threshold = ciCOHs_sorted[i-1]
    
    print("corrected threshold = " + str(corrected_threshold))
    
    return corrected_threshold, mu, std
        

###  Normal Distribution

In [ ]:
def threshold_ciCOH_normal(ntimes, ntrials, ntemp, alpha = 0.05, ploton = True):
    """
        using normal distribution to simulated the no connections and identify the threshold for has connection
        
        @paras:
            ntimes: the repeated time (can be set nchns * nchns)
            ntrials: the number of trials 
            ntemp: the length of the temporal data
            alpha: the critirial (5%, or 1%)
            ploton: show a figure if True
        
        @return:
            threshold
    """

    ## generate time series with Normal Distribution (no connection)
    mu, sigma = 0, 1
    sigs1, sigs2 = normal(mu, sigma, (ntimes, ntrials, ntemp)), normal(mu, sigma, (ntimes, ntrials, ntemp)) 


    ## calculate the ciCOH of the simulated time series
    ciCOHs = []
    for timei in range(ntimes):

        sig1, sig2 = sigs1[timei, :, :], sigs2[timei, :, :]

        # calculate ciCOH
        ciCOH = np.mean(ciCoherence_acrosstrials(sig1, sig2))
        ciCOHs.append(ciCOH)

        del ciCOH, sig1, sig2 


    ## return the threshold
    # Fit a normal distribution to the data:
    mu, std = norm.fit(ciCOHs)

    if alpha == 0.05:
        threshold = np.around(mu + 2*std, decimals=4)
    if alpha == 0.01:
        threshold = np.around(mu + 3*std, decimals=4)

        
    ## plot the ciCOH distribution of the simulated data 
    if ploton:
        _plot_ciCOH_distributions(ciCOHs)
        
    return threshold

## Test Part

In [ ]:
if False:
    thres = threshold_ciCOH_sin(ntimes = 1000, ntrials = 100, ntemp = 500, f = 20, t = 1, alpha = 0.05, ploton = True)
    
    
    print(thres)